# Model 4: The 0/1 Knapsack Problem

Given a set of items, each with a weight and a value, determine which items to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible. The "0/1" binary qualifier denotes that each item must be entirely accepted or rejected, that is, you cannot subdivide an item.

## Modeling components

### Sets

- $J$: Set of items indexed by $j$, $j \in J$

### Parameters

- $w_j$: Weight of item $j$ for each $j \in J$
- $v_j$: Value of item $j$ for each $j \in J$.
- $W$: Maximum weight capacity of the knapsack.

### Decision variables:
$$x_j= \begin{cases} 1, & \text{if item $j$ is included in the knapsack,} \\ 0, & \text{otherwise.} \end{cases}$$

## The integer programming formulation

$$
\begin{array}{rl}
    \max  & \sum_{j\in J} v_j\, x_j \\[5pt]
    \text{s.t.} & \sum_{j\in J} w_j\, x_j \leq W \\[5pt]
    & x_j \in \{0, 1\},\ \forall j\in J
\end{array}
$$

## The knapsack caper: A story of greed, math, and risk

Your customer is an international jewel thief, who has a plan to heist a jewelry store this week. The store has eight different types of jewels, each with a different value and weight. The table below displays the value and weight of each jewel type.

$$
\begin{array}{|l|rr|}
\hline
    \text{Jewel Type} & \text{Value (kCAD)} & \text{Weight (g)} \\
\hline
    \text{Opal}       & 11          & 150        \\
    \text{Turquoise}  & 9           & 150        \\
    \text{Garnet}     & 13          & 60         \\
    \text{Jade}       & 10          & 100        \\
    \text{Onyx}       & 8           & 125        \\
    \text{Coral}      & 7           & 100        \\
    \text{Lapis}      & 3           & 50         \\
    \text{Agate}      & 5           & 80         \\
\hline
\end{array}
$$

The thief has a special bag that can stretch to fit any volume of jewels, but it can hold up to 600g safely. Exceeding this threshold could cause structural failure, rupturing the bag and spilling its contents. Due to security factors, the thief can only take one jewel of each type. However, with eight types available and a strict weight limit, tough decisions must be made. As the thief's most trusted advisor, he is relying on your analysis to determine the optimal set of jewels to steal, maximizing payoff while ensuring a clean getaway. Can you assemble the right package of plunder?

## The concrete Pyomo model

### Import required libraries and create the model object

In [ ]:
!pip install gurobipy pyomo

solver_options = {
    "WLSACCESSID": "...",  # your WSL access id (string)
    "WLSSECRET": "...",  # your WSL secret (string)
    "LICENSEID": ...,  # your license id (integer)
}

In [ ]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

mod = pyo.ConcreteModel(name="binary_knapsack")

### Define the components of the model

#### Sets:

In [ ]:
# set of jewel types:
jewels = {"Opal", "Turquoise", "Garnet", "Jade", "Onyx", "Coral", "Lapis", "Agate"}

#### Parameters:

We use dictionaries to map each jewel to its value and weight.

In [ ]:
values = {
    "Opal": 11,
    "Turquoise": 9,
    "Garnet": 13,
    "Jade": 10,
    "Onyx": 8,
    "Coral": 7,
    "Lapis": 3,
    "Agate": 5,
}

weights = {
    "Opal": 150,
    "Turquoise": 150,
    "Garnet": 60,
    "Jade": 100,
    "Onyx": 125,
    "Coral": 100,
    "Lapis": 50,
    "Agate": 80,
}

W = 600  # the knapsack capacity

#### The decision variables, the objective, and the constraint

To create a list of variables in a Pyomo model, you can use the index set as the first argument in the `pyo.Var()` function. The index set specifies the set or list of indices that define the variables.

In [ ]:
# create a dictionary of decision variables, using index set J:
mod.x = pyo.Var(jewels, domain=...)
# Note that the jewels set is provided as the first argument

In [ ]:
# objective function:
expr = sum(values[j] * mod.x[j] for j in jewels)
mod.obj = pyo.Objective(expr=expr, sense=pyo.maximize)

# constraints:
expr = sum(weights[j] * mod.x[j] for j in jewels)
mod.capacity = pyo.Constraint(expr=expr <= W)

Inspect your created objects:

In [ ]:
# using the built-in pprint() method for pretty printing

mod.x.pprint()
mod.obj.pprint()
mod.capacity.pprint()

### Solve the model

In [ ]:
opt = SolverFactory("gurobi", solver_io="python", manage_env=True, options=solver_options)
result = opt.solve(mod, tee=False)

### Display and interpret the results

In [ ]:
# calculate the total value
total_value = pyo.value(mod.obj) * 1000
print("Objective value =", total_value, "CAD")

# calculate the total weight
total_weight = sum(weights[j] * pyo.value(mod.x[j]) for j in jewels)
# or simply, `total_weight = pyo.value(mod.capacity)`
print("Total weight    =", total_weight, "g")

# Display the selected jewels
print("Selected Jewels =", end=" ")
# print the jewels with non-zero values in the variable dictionary
for j in jewels:
    if pyo.value(mod.x[j]) > 0.5:
        print(j, end=", ")

### Exercise

1. Consider a specific case: If you have the option to select either the "Opal" or the "Garnet" jewel, or neither of them, what would be the optimal set of jewels to steal?

In [ ]:
# Create a set containing the items that cannot picked together:
not_all_items = {...}

# Create constraint expression
expr = sum(mod.x[j] for j in ...)

# Create constraint
mod.con1 = pyo.Constraint(expr=...)

In [ ]:
# Solve the model
result = opt.solve(mod)

# See the results
total_value = pyo.value(mod.obj) * 1000
print("Objective value =", total_value, "CAD")

print("Selected Jewels =", end=" ")
for j in jewels:
    if pyo.value(mod.x[j]) > 0.5:
        print(j, end=", ")

2. What if the thief is required to steal at least five items that weigh 100g or more?

In [ ]:
# Create a set containing the items that weigh 100g or more
heavy_items = {j for j in jewels if ...}

# Create constraint expression
expr = sum(mod.x[j] for j in heavy_items)

# Create constraint
mod.con2 = pyo.Constraint(expr=...)

In [ ]:
# Solve the model
result = opt.solve(mod)

# See the status of the solution
print("solution status =", result.solver.termination_condition)

# Appendix: How to create tables in Jupyter

## Method 1: Using Markdown syntax

Markdown is a simple and lightweight markup language that allows you to format text using plain text syntax. You can use Markdown syntax to create tables in Jupyter notebooks by using pipes (`|`) and dashes (`-`) to separate the cells and rows. For example, you can write:

```markdown
| Jewel Type | Value (CAD) | Weight (g) |
|------------|-------------|------------|
| Opal       | 11          | 150        |
| Turquoise  | 9           | 150        |
| Garnet     | 13          | 60         |
| Jade       | 10          | 100        |
| Onyx       | 8           | 125        |
| Coral      | 7           | 100        |
| Lapis      | 3           | 50         |
| Agate      | 5           | 80         |
``````
This will produce the following table:

| Jewel Type | Value (CAD) | Weight (g) |
|------------|-------------|------------|
| Opal       | 11          | 150        |
| Turquoise  | 9           | 150        |
| Garnet     | 13          | 60         |
| Jade       | 10          | 100        |
| Onyx       | 8           | 125        |
| Coral      | 7           | 100        |
| Lapis      | 3           | 50         |
| Agate      | 5           | 80         |

Markdown syntax blends nicely in the text and is easy to read and write. However, it has some limitations, such as the inability to align the text in the cells, or to add borders or colors to the table.

## Method 2: Using LaTeX syntax

LaTeX is a powerful and widely used document preparation system that allows you to create professional-looking documents with complex formatting and typesetting. You can use LaTeX syntax to create mathematical equations and tables in Jupyter notebooks by using the array environment and specifying the column alignment and the vertical lines. For example, you can write:

```latex
$$
\begin{array}{|l|rr|}
\hline
    \text{Jewel Type} & \text{Value (kCAD)} & \text{Weight (g)} \\
\hline
    \text{Opal}       & 11                  & 150               \\
    \text{Turquoise}  & 9                   & 150               \\
    \text{Garnet}     & 13                  & 60                \\
    \text{Jade}       & 10                  & 100               \\
    \text{Onyx}       & 8                   & 125               \\
    \text{Coral}      & 7                   & 100               \\
    \text{Lapis}      & 3                   & 50                \\
    \text{Agate}      & 5                   & 80                \\
\hline
\end{array}
$$
```
This will produce the following table:
$$
\begin{array}{|l|rr|}
\hline
    \text{Jewel Type} & \text{Value (kCAD)} & \text{Weight (g)} \\
\hline
    \text{Opal}       & 11                  & 150               \\
    \text{Turquoise}  & 9                   & 150               \\
    \text{Garnet}     & 13                  & 60                \\
    \text{Jade}       & 10                  & 100               \\
    \text{Onyx}       & 8                   & 125               \\
    \text{Coral}      & 7                   & 100               \\
    \text{Lapis}      & 3                   & 50                \\
    \text{Agate}      & 5                   & 80                \\
\hline
\end{array}
$$
LaTeX syntax provides more control over text alignments and decoration, and allows you to customize the appearance of your table with various options. However, it is more complex and verbose than Markdown syntax, and requires some basic knowledge of LaTeX commands and syntax.